In [16]:
import os,sys

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score, recall_score, roc_auc_score, confusion_matrix
from collections import Counter
import scipy.io
import random
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from loc_utils import VolumeVisualizer,roc






In [17]:
# Define a random seed for the entire session
random_seed = 1995

# Set the random seed for numpy, random, and sklearn
np.random.seed(random_seed)
random.seed(random_seed)
from sklearn.utils import check_random_state
check_random_state(random_seed)

RandomState(MT19937) at 0x259CFA84C40

In [15]:
'''The features for this demo are stored as part of a .mat file that places patient and feature as a dataframe. along with the labels.
For this example the training and testing splits have alread been defined   

Inflammation
training=[1	8	9	12	16	22	26	34	35	37	39	42	47	49	51	2	7	13	14	18	20	21	23	30	32	33	36	38	40	41	43	44	45	50]
testing=[31	11	46	29	5	10	27	19	48	15	3	24	25	28	4	6	17]
"H:\My Drive\Research\MREcrohnsproject\Data\Florian_processed\3DFeats\Stack\Strict_Crohns_TI_featstack_multiROI-v3_wc.mat"
                featstack_TI=featstack_mre_SRmed(training,(1:end-3));
                labels=featstack_mre_SRmed(training,(end-2:end));

Fibrosis

 training=[3,16,17,20,24,27,31,34,36,40,41,46,49,51,1,4,6,7,8,10,11,13,14,15,18,21,22,26,29,30,32,33,38,45]
 testing=[39,50,43,9,35,44,12,25,23,48,42,37,5,19,47,28,2]
"H:\My Drive\Research\MREcrohnsproject\Data\Florian_processed\3DFeats\Stack\Strict_Crohns_TI_featstack_multiROI-v3_wc.mat"
                 featstack_TI=featstack_mre_SRmed(training,(1:end-3));
                labels=featstack_mre_SRmed(training,(end-2:end));
'''


ModuleNotFoundError: No module named 'loc_utils'

In [15]:
mat_data=scipy.io.loadmat(r'H:\My Drive\Research\MREcrohnsproject\Data\Florian_processed\3DFeats\Stack\Strict_Crohns_TI_featstack_multiROI-v3_wc.mat')
print(scipy.io.whosmat(r'H:\My Drive\Research\MREcrohnsproject\Data\Florian_processed\3DFeats\Stack\Strict_Crohns_TI_featstack_multiROI-v3_wc.mat'))



[('checklist2', (950, 1), 'cell'), ('statnames', (0, 0), 'cell'), ('featstack_mre_whole', (51, 2167), 'double'), ('featstack_mre_SRmode', (51, 2167), 'double'), ('featstack_mre_SRmed', (51, 2167), 'double')]


In [23]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV

# Extract features and labels from the mat_data
featstack_mre_SRmed = mat_data['featstack_mre_SRmed']
print(featstack_mre_SRmed.shape)
labels=featstack_mre_SRmed[:,-3:]#last three columns are the labels 
print(labels)
featstack_mre_SRmed=featstack_mre_SRmed[:,:-3]#remove the last three labels


# Define training and testing indices
training_inflammation = [1, 8, 9, 12, 16, 22, 26, 34, 35, 37, 39, 42, 47, 49, 51, 2, 7, 13, 14, 18, 20, 21, 23, 30, 32, 33, 36, 38, 40, 41, 43, 44, 45, 50]
testing_inflammation = [31, 11, 46, 29, 5, 10, 27, 19, 48, 15, 3, 24, 25, 28, 4, 6, 17]
training_inflammation = [i - 1 for i in training_inflammation]
testing_inflammation = [i - 1 for i in testing_inflammation]


training_fibrosis = [3, 16, 17, 20, 24, 27, 31, 34, 36, 40, 41, 46, 49, 51, 1, 4, 6, 7, 8, 10, 11, 13, 14, 15, 18, 21, 22, 26, 29, 30, 32, 33, 38, 45]
testing_fibrosis = [39, 50, 43, 9, 35, 44, 12, 25, 23, 48, 42, 37, 5, 19, 47, 28, 2]
training_fibrosis = [i - 1 for i in training_fibrosis]
testing_fibrosis = [i - 1 for i in testing_fibrosis]


# Split the data into training and testing sets
X_train_inflammation = featstack_mre_SRmed[training_inflammation,:]
y_train_inflammation = labels[training_inflammation,0]
X_test_inflammation = featstack_mre_SRmed[testing_inflammation,:]
y_test_inflammation = labels[training_inflammation,0]

X_train_fibrosis = featstack_mre_SRmed[training_fibrosis,:]
y_train_fibrosis = labels[training_inflammation,1]
X_test_fibrosis = featstack_mre_SRmed[testing_fibrosis,:]
y_test_fibrosis = labels[training_inflammation,1]


print(X_train_inflammation.shape)



(51, 2167)
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 0. 1.]
 [1. 0. 1.]
 [1. 1. 1.]
 [1. 0. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 0.]
 [0. 1. 1.]
 [0. 0. 0.]
 [1. 0. 1.]
 [0. 0. 0.]
 [0. 1. 1.]
 [0. 1. 1.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 0.]
 [1. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 0. 1.]
 [1. 1. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 0.]
 [1. 1. 1.]
 [1. 0. 1.]
 [0. 0. 0.]
 [1. 1. 0.]
 [0. 1. 1.]
 [1. 1. 0.]]
(34, 2164)


In [24]:
# Define the Random Forest model
rf = RandomForestClassifier()

# Perform Recursive Feature Elimination with Cross-Validation (RFECV)
# Perform Recursive Feature Elimination with Cross-Validation (RFECV)
rfecv = RFECV(estimator=rf, step=1, cv=StratifiedKFold(3), scoring='roc_auc')
rfecv.fit(X_train_inflammation, y_train_inflammation)

# Get the selected features
selected_features = rfecv.support_

# Perform hyperparameter search using GridSearchCV
param_grid = {
    'n_estimators': [51,101, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=StratifiedKFold(3), scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train_inflammation[:, selected_features], y_train_inflammation)

# Train the final model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train_inflammation[:, selected_features], y_train_inflammation)

RandomForestClassifier(n_estimators=101)

In [ ]:
# Evaluate the model on the test set
y_pred_inflammation = best_rf.predict(X_test_inflammation[:, selected_features])
accuracy = accuracy_score(y_test_inflammation, y_pred_inflammation)
f1 = f1_score(y_test_inflammation, y_pred_inflammation)
recall = recall_score(y_test_inflammation, y_pred_inflammation)
roc_auc = roc_auc_score(y_test_inflammation, y_pred_inflammation)
conf_matrix = confusion_matrix(y_test_inflammation, y_pred_inflammation)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Recall: {recall}')
print(f'ROC AUC Score: {roc_auc}')
print(f'Confusion Matrix:\n{conf_matrix}')

